# IBM Cloud Hyper Protect Secure Build Server and BYOI lab

This lab will introduce you to two features of the IBM Cloud Hyper Protect Virtual Servers offering:

1. Secure Build Server
2. Bring Your Own Image, or BYOI

The initial introduction of **_IBM Cloud Hyper Protect Virtual Servers_** (HPVS) offered an Ubuntu image which you could then tailor to your requirements, but if you wanted to replicate your changes onto other HPVS instances, you would have to repeat the custom configurations on each additional HPVS instance you created.  Automation tools like Ansible could help with that process, but wouldn't it be nicer if you could just deploy your custom-built Docker image, that you only had to configure once, onto an HPVS instance?  That's the convenience that the *BYOI* feature offers. It offers the ability to deploy a digitally signed Docker image into an HPVS instance.

You can deploy Docker images that are digitally signed by you or some other person or organization that you (hopefully!) trust. In the case of images you build and sign, the _Secure Build Server_ feature lets you build, and digitally sign, a Docker image within an HPVS instance called a _secure build server_ that offers the confidentiality, integrity and auditability protections inherent in IBM Cloud Hyper Protect Virtual Servers.  

In this lab you will perform the main use case targeted by these two features working together:

1. You will deploy your own secure build server in an HPVS instance
2. You will use this secure build server to securely create and digitally sign a Docker image that contains a sample application. For this lab we will use a sample _secure bitcoin wallet application_ provided by IBM for demonstration purposes.
3. You will deploy this Docker image that you've built in a second HPVS instance and then run the sample secure bitcoin wallet application contained within the image. You will configure your HPVS instance to use an **_IBM Cloud Hyper Protect Crypto Services_** (HPCS) instance for key generation.  Your HPCS instance provides the security of a _FIPS 140-2 level 4_-certified _Hardware Security Module_, which is the strongest level of FIPS 140-2 certification commercially available.

You will be using an open-source application, called _Jupyter Notebook_, to run this lab. This application allows you to enter commands and see the results of these commands. You are likely reading this within Jupyter Notebook right now.  You will also open a terminal within Jupyter and enter some commands from the terminal. The large majority of the commands will be run from within the Jupyter notebook while a small number of commands will be run from the terminal. Those commands that prompt for user input are run from the terminal, while the vast majority, which don't prompt for user input, are run from within Jupyter Notebook.

## Initial login to IBM Cloud via the ibmcloud command line interface (CLI)

### Step 1.1 Open a terminal with the Jupyter Launcher

You should have two tabs open in the browser that is within your Red Hat Enterprise Linux 8.3 system from which you are taking this lab:

    1. The tab you are using right now to read this lab
    2. The Jupyter launcher tab, from which you launched this lab.

Go back to the Jupyter Launcher tab.  Near the upper right in that tab you will see a button that says *New* with a dropdown arrow. Click on the *New* button and select *Terminal*. This will open a new tab that has a terminal window that you will use to enter a small number of commands from this lab that require interactive user input.

![Choose Terminal](Bitcoin_Wallet_Images/OpenNewTerminal.png)

When you have done that, come back here to continue with the instructions.

---

### Step 1.2 Use the terminal tab to log in to IBM Cloud

Navigate to your terminal tab, and from there, enter the command `ibmcloud login -r YOUR-REGION --sso`, substituting one of the following regions for YOUR-REGION:  `us-east`, `au-syd` or `eu-de`, which are for United States (East), Sydney, Australia, and Frankfurt, Germany, respectively.  

For your convenience the proper command for each region is listed. From the three commands listed below, copy the one applicable for your region, and paste it into the terminal tab and run the command:

`ibmcloud login -r us-east --sso`

`ibmcloud login -r au-syd --sso`

`ibmcloud login -r eu-de --sso`

---

### Step 1.3 Follow the prompts to complete the login

Follow the instructions to log in to your IBM Cloud account.  The login process from the terminal should be like this:

You will first be given a *Y/n* prompt, reply *Y*.

At that point, the login command tries to open a browser tab and display a temporary one-time code for you. 
In order to get this code to use to log in with the CLI, you may need to log in to the IBM Cloud Web UI with your browser. If so, follow any necessary login prompts. You should eventually be given a page with a the one-time code.  Click on that code to put it in your clipboard, and paste it at the input prompt back in your terminal tab.

When successful, your terminal window will look like this, but with your account information of course.: 

```

API endpoint:      https://cloud.ibm.com
Region:            us-east
User:              silliman@us.ibm.com
Account:           Barry Silliman's Account (1e963a246cc69a44df65e277e14239d5) <-> 1996902
Resource group:    No resource group targeted, use 'ibmcloud target -g RESOURCE_GROUP'
CF API endpoint:
Org:
Space:
```

When that is complete, come back to this tab and continue in this notebook. Leave the terminal tab open, you will need it later.

---

### Step 1.4 Target the default resource group

---

#### Jupyter Notebook tip

The text you are reading now is in a type of Jupyter notebook cell called _markdown_.  It is intended for instructions, and includes text and possibly images.

Commands that you will enter (with the exception of the small number of commands, such as the `ibmcloud login -r YOUR-REGION --sso` above, that are run in the terminal tab) will be in a second type of Jupyter notebook cell called _code_.  At the top of this page you can see a dropdown box that will show you whether or not you are in a markdown cell or a code cell.

---

You can see from the above output that no _resource group_ is targeted when you log in. The command in the code cell below will target the _default_ resource group. Resource groups are logical buckets that you can define in order to categorize your IBM Cloud resources as you wish.  We will simply use the _default_ resource group which exists already.

Click on the code cell below that contains the command `ibmcloud target -g default`. You should see _code_ in the dropdown box at the top. 

Now that you have the code cell below highlighted, click the _Run_ button from the menu above, and that command will be run, and its output will be shown.

This will target your _default_ resource group. See the debugging tip below the code cell if this command does not work for you.

In [ ]:
! ibmcloud target -g default

#### Debugging tip

Some IBM Cloud users may not have a _default_ group but instead have a _Default_ group, with an upper case _D_.  If the above command fails, try using *Default* instead of *default*.  You can do that by just clicking in the code cell above and changing the _d_ to _D_ in default.  Then run the code cell again. If that still doesn't work, ask an instructor for help.

---

### A note about running Jupyter Notebook code cells

Observe that to the left of the code cells are square brackets with empty contents. When you run a code cell, an asterisk (*\**) appears between the square brackets.  (If the command runs fast enough you may not even notice the appearance of the asterisk). When the command finishes, a number appears between the square brackets.  During this lab, some of the commands may take several seconds to run, but no command in this lab should take too long. Each time that you run a code cell, ensure that you wait for that code cell to complete execution before continuing with the lab. If the asterisk remains in between the square brackets for more than thirty seconds, ask an instructor for help—none of the commands in this lab should take that long.

The number shown between the square brackets upon command completion is incremented by one each time you run a code cell.  The numbers are helpful in ensuring that you have run each code cell in the correct order.

---

## Create an ssh key for the lab and add it to your GitHub account

### Step 2.1 Create an ssh key pair in your lab system

As part of this lab, you will need to create a key pair that you will then define in your personal GitHub account settings so that this key pair can be used to clone GitHub repositories.

**Note:** If you are an experienced GitHub user and already have ssh keys defined to your GitHub account, you should still create a new key for this lab and remove it from your account when the lab is over, for security reasons. **What is the security concern?** The lab instructors have access to your lab system in order to be able to help with debugging.  If the lab instructors were malicious, they could copy your ssh key, including the private key portion of the key pair.  They are not malicious—they are well-intentioned and kind. But to be completely secure, you should create a new key per the instructions in this step, and then remove it from your GitHub account when you have completed the lab. (The cleanup section of this lab helps you with the key removal from your GitHub account).

Run the code cell below to create a key pair in your lab workspace.

In [ ]:
! ssh-keygen -t rsa -b 4096 -f '/home/ibmuser/.ssh/id_rsa' -N ''

---

### Step 2.2 Copy your public key to the clipboard

The above command created a key pair (a private part that you keep secret and a public part that can be shared safely with the world).  You need to add the public part to your GitHub account settings, so that this key pair can be used to do a clone of the GitHub repository of the sample secure bitcoin wallet application from within the Secure Build Server container.

Run the code cell below to list the public key, and copy this public key into your clipboard:

In [ ]:
! cat ~/.ssh/id_rsa.pub

Copy all of the output from the above `cat` command into the clipboard, and go to [this link](https://github.com/settings/ssh/new) to add this key to your GitHub account. When you get to that page (you may have to log in to GitHub first), put anything you want in the _Title_ field and paste the above output (which is your public key) into the _Key_ field and then click *Add SSH key*.  

**Note:** if you use a password manager and need to use copy and paste to log in to your GitHub account, go ahead and log in and then copy and paste the above output.

---

### Step 2.3 add the GitHub public keys to your _known\_hosts_ file

Run the following code cell which gathers the public SSH keys of github.com and puts them in your lab workspace's _known\_hosts_ file. This is done only to eliminate a prompt about trusting the github.com host in the code cell after this one.

In [ ]:
! ssh-keyscan -H github.com >> "${HOME}/.ssh/known_hosts"

---

### Step 2.4 Verify you have added the key to your GitHub account correctly

Run the code cell below to verify that you have correctly added your SSH key pair to your GitHub account.  You should receive a message that says _Hi \<your GitHub username\>! You've successfully authenticated, but GitHub does not provide shell access._  If you received this message, continue in the lab.  If not, ask an instructor for help.  

In [ ]:
! ssh -T git@github.com

---

### Step 2.5 Create a personal access token for your GitHub account

The build of the sample secure bitcoin wallet application requires that a GitHub personal access token be provided to it. Even if you already have a GitHub personal access token, you are advised to create a new one for this lab. Log in to your GitHub account — you may still be logged in from _Step 2.1_ — and follow steps 1-9 (i.e., you can skip step 10) in [these instructions](https://docs.github.com/en/github/authenticating-to-github/creating-a-personal-access-token#creating-a-token) in the _Creating a token_ section.

Ensure that you save the value of your newly created personal access token, as you will need it later on in the lab.

## Create an IBM Cloud IAM API Key for this lab

### Step 3.1 Create an IAM API Key

**Note:** _IAM_ stands for _Identity and Access Management_. _API_ stands for _Application Programming Interface_.

Run the command in the _code_ cell below in order to create an API key for your user ID. This will be used in the lab for you to store your custom-built Docker image into the IBM Cloud Container Registry.

*Note:* If you are an experienced IBM Cloud user and already have some API keys, create a new one anyway, for usage just for this lab, and delete it when the lab is over. (The cleanup section of the lab will help you to delete it). Do not use it for other purposes. This advice is given in order to mitigate the same lab-specific security concern described in the previous section on creating the SSH key.

If, by chance, you already have an API key named _myapikey_, choose a different name for this lab.  You can do that by double-clicking on _myapikey_ in the code cell below to highlight it and then change its name before you run the code cell.

In [ ]:
! ibmcloud iam api-key-create myapikey -d "API key for SBS tutorial"

---

### Step 3.2 Save the value of the API key

The output of the above code cell will contain an _API Key_ field.  Copy the value of this field and save it someplace safe for the duration of this lab.  (The output containing this should remain within the Jupyter Notebook unless you clear the output, but save it elsewhere as well just in case the output is accidentally cleared). There is no way to retrieve this value later from the _ibmcloud_ CLI.  The _API Key_ field should be treated as a password, as that is essentially what it is.

---

## Lab setup concerning the IBM Cloud Container Registry

### Step 4.1 Set your IBM Cloud Container Registry region

Run *one* of the three code cells below, picking the IBM Cloud Container Registry region name that corresponds with the region that you targeted when you logged in via the *ibmcloud* CLI at the beginning of the lab in *Step 1.2*.  The comment at the end of each command helps you pick the right code cell:

In [ ]:
! ibmcloud cr region-set us-south   # run this one if you logged in to the us-east region

In [ ]:
! ibmcloud cr region-set ap-south   # run this one if you logged in to the au-syd region

In [ ]:
! ibmcloud cr region-set eu-central # run this one if you logged in to the eu-de region

### Step 4.2 Create a container registry namespace

The code cell below contains a command to add a namespace to the IBM Cloud Container Registry that will be owned by you. You can try to run the command as shown, but it will definitely fail—go ahead and try it, it won't hurt anything.  It will tell you that the _secureimages_ namespace already exists.  Change _secureimages_ to something unique by adding your initials and maybe a number or some other unique string to the namespace.  Keep trying the below code cell, changing what was originally _secureimages_ to a different name, until you get a message that indicates success.  For example, the author appended his initials, i.e., he used _bjs-secureimages_ and that worked for him.

**Note:** If you have used IBM Cloud Container Registry previously, you may already have a namespace. Feel free to either use that or create a new one for this lab. There are no security concerns for using one of your existing namespaces. If you run the below code cell with a namespace you already own, the output message will indicate that you already own it. Again, that's fine if you want to use a namespace you already own.

**Jupyter Notebook tip:** If you click the *Run* button above after you've highlighted the code cell below (or any cell for that matter) it will attempt to run the code and then move to the next cell.  If you need to retry because the name you chose was already taken, just click in the cell again and try again.

In [ ]:
! ibmcloud cr namespace-add secureimages

---

## Create your Secure Build Server HPVS instance

### Step 5.1 Download the Secure Build command line interface (CLI):

In order to set up your own Secure Build Server within an HPVS instance, you will use a command line interface (CLI) that is provided by IBM.  This CLI is implemented in Python and is publicly available (including source code) in a GitHub repository, which you will download now into your lab environment when you run the following code cell:

In [ ]:
! git clone git@github.com:ibm-hyper-protect/secure-build-cli.git

---

### Step 5.2 Change the working directory for the remainder of the lab

Up to now, your notebook has been running from the _/home/ibmuser/code/ibm-cloud-hpvs-sbs-byoi-lab_ directory. For the remainder of the lab, we want to work from the directory that was just created by the _git clone_ command from the prior code cell.  Our new desired working directory is a subdirectory of our current working directory.

Run the next code cell in order to change directories to the _secure-build-cli_ directory that was created by the `git clone` command that you just ran.

In [ ]:
%cd secure-build-cli

---

### Step 5.3 Install Python packages required by the Secure Build CLI

There are some python packages required by the Secure Build CLI, and running the following code cell will install these packages using *pip3*, the Python 3.x-based package installer:

In [ ]:
! pip3 install -r requirements.txt

---

### Step 5.4 Create the Secure Build Server registration file

Part of the "BYOI" workflow involves creating a _secure registration file_, which contains secrets and metadata that can only be decrypted by the running image.  When you build an image later on in the lab, you will download a secure registration file for the image you will have built, and you will pass this file as an argument to the _ibmcloud hpvs_ command that you will use to create an HPVS instance built from your new image. 

The Secure Build Server itself is deployed securely within an HPVS instance with the help of its own secure registration file.  The code cell below will place this file into your lab environment workspace with the name `secure_build.asc`.

This file is posted publicly on the public documentation page for IBM Cloud Hyper Protect Virtual Servers, but the Secure Build CLI repository on GitHub does not contain it.  That's why you are creating it now.  This file is encrypted, and only the Secure Build Server can decrypt it.  IBM Cloud administrators cannot access your secure build server at all, and you can only access it through the Secure Build CLI, which limits your access to specific functions. Therefore, by design, the decrypted contents of this file cannot be obtained by any human.

**Note:** The code cell below will not produce any output, because it copies the encrypted message into a file.  But you know the command ended if a sequence number is placed within the square brackets to the left of the code cell. 

In [ ]:
%%bash
cat <<EOF > secure_build.asc
 -----BEGIN PGP MESSAGE-----

 hQIMA4AFxsw8YZ3rARAAnR1PB+ij0LNdueQIxtzIVnJ2ogQdh7+akYLC/V85u8Xh
 NKZmxsjwzVsCISZOSwvn/BbylonBPXG+c99gI60ChA+vjpCmXqU7dASWTjUyBfYZ
 piu6ktHRpQrsAPPZl2o+17yjFYnlRpG2FkVTfEMMAdGbC4j+g345RgF3trBgsMdF
 rvaJa2mbsekau9I076hNHKgm02ar3VHET8Y1sMGmij87cgmvKipqoyF/TIfv8C0c
 fK2CX4ty8DRMINPSojGiDTR15JBd+QwdeW9cO1fKEczKMiXiNcDawo8X0r0ICHKs
 +QssrlOwn0XG7QzQhW6DOdAjWmu5QnlCAs5+yJ47vMfLL5g+84S2Bm2Lrn6fbeVc
 cBcBZr79Bbp4uDsF8OMVfrEl2Kv3fQnTbnA3bL7k1W/ABU5ytPYnBZDo/h45xq8z
 Cpw3lVNnMXqN2+i4Dc3dF+qtSMOo3nlmEcMxfhUDooWAyXX4/cysjU9P1TCs4OsA
 inov8GM7V2nYXFTKo9rKfS7UMiqsLAKwpw6YvP2tZUoNk/G8Wyb4cjB052R28MSG
 H8VBbIouKyJhDuSJk4J2pwWCwQYFcE5CTGzIStWZpu5CdYr9TMlVNZJ1jrwp0a+o
 wlFA3WHGYczSWQa0smc8U7L/jcOVAtV0u2Q+5g3AwJUy/ogZ7WfynJ+t7H3a0WjS
 7AGCRvaiPyVroBJuaGOwFHAXE19u5/I4xHgYpDaaQlbcX2m3MVdi8N96UHnTXKvH
 QLPi7IeciT2/56eYULzynb8PuFyQbmS8WSeSJmjUB0wZB12Ys4xUAUObuHwzLgk0
 2zPRJjRM7NrDw824yJGmSDr+I9E30we8TMJX4ZGP2a2q8RrXlxLgUqgMU8QKe+wx
 p96HmaNC3L4Vahhr7/U2OQ638r/ow++xOvbhVn3X8uFmA5I9PW92AdpqtaRuKTtw
 a1+63K32ctcWaEH7DUGfBmWaqpD/QZbE/SFhMT+sHEuLXgOP/dbTdWSGtW4upkGp
 bIwrcU30gZ05J1O8s2kRc+K2m4Ty44AILT0ddhGCGJicJDLAl0JNryFP4Eazs/Q8
 WawQN6sl6Hr7f2QDjJcaxUtZ3uO7/r46AjRap8j+UP3GJEoDf2Ry1/lVO1xGTEWl
 KNo4db5Xkuxq1xNJPCigpv206eH0Fuk8G3S5EK+h7PUmZ1aQgE8GzZJs21FqI+gY
 WGqXqp98rLYFX4IjEm7be8kTWlB0QZCtfA1cqtWNRy0chrak+XYbgPPptai26Eox
 hIAInWEkIzLWC/4t7WIgIOOWwKyMdeNHj0iAlihGToqtmqtcE5pj+CyRqzA7uHUb
 w1H4+q0w92Tcf6d7+xKrZ1OuJZ+5v7j/S1ZhLCMWC0yQzEZCCK4pc8D3I6ZoMcRK
 nlgJXKznjI27A6ZwXi8yBJQu+TvJIwIpP2nTF9LK76bktsQYAAS/zcxSBlCSMkeu
 gqeE82sf32j0JvAdplbHD0uq9fQeJRJQ2nMae7qYwIy6d2XMZMWCKsKxKm9BB8bU
 qpmtGIGjHyTZTCTh/BVsaPL++jCKhLq4W/8J5C/58hhfR4gH/V6sDmmAQb85wotT
 tGnZzVDDqL1zk1Qou7+6tndNdOTWVmydp70xkJE393CngLiuzDCoZuFFC8F3XZXW
 Qx+2+upbmhoqg9QwnMZPjemb1sesMHPUJn+wRlpSbiYiaYA9qxyozuQBcq2C8zP7
 Y8Kpbsajve4dKvXz4p9S1jsRzuE1wQ0/3WjUawYQQiv08v9yurhUB2yWilfv0ypX
 1TrISFaCkHBhDK10m/2VzGDR9lr5hPcHL8KuM3Mi5XqQagvmPrdLC4UR0EP7NveX
 zkg4GkBgrQpV9Sxq9x6zndSymdCpmYVUnBRZtS94qAhfpr3riMkL1EUWvzlVp6um
 MbMBKjWhtBpvLNTeFR2qtE5FjFyeY0k08MT6W5MvtJ4tljOX65kdXDeBa+HZD+GI
 Ae3UMTrT1WR0hOcOa6nGkpyidFGyL1wZBF4AgbCsh7jQ/tiYm1oQh8lRFBvm4R9S
 dJpB7rX92QCEvcfROlXBUCGmbYxl4L8SFdqdlS7HjGLi5YYI9JeT4FylAd2r0Ehm
 HSuuG0Xlvedn34WDTrYOMoLl28Lha4tHl8WKaY4XqnNJ9JNqEvqRdVs/f5LcY6ja
 PTo4M8x51ZqZKE8WXtJeHQF0kA8I5i65TA/N/56tnCidVciM6oornTocsxMdMsW4
 y2NvsgS+vkMh+AhkaHR+rTu+FsbRwlWdd16NtLSNS4Q6H8n6+K8nzyB7q9D0f2uO
 1EODUQrM+08LRKL1tQJQSyhMcj13X8p9BqEYmAaGU8TrNRHx1+Yg+5iyTrAOSPqQ
 7oQTCojdrkE0MEQVYTvakZJs7yFYMpuO8Nq97DJg5++ntutUU0dMt0LNqfN/3xkY
 GdUodrb/lKIJSZAquFtm0zwjKW3d7qLWA9WWblLNz5yvJHSm9LSoEAe4gYnNfv1v
 VEHwjp5u0aWWWFE8Vwh6c5SeOZNbjfR9xwxdQJ3SJS59RHuWsv/A1CNR4P77Ow86
 Z4SfMJDxh7QQJ5wQnyIJdL0ESgPqDcK0yiWPzuQTuBweadlTNpOKApr8E29lfmX4
 jkRDcEDELwQEFqb2cISzo6Y+lC6aJvRnZnbaCb6zj3IGbpqFD8PXwiwTkV9yGfSY
 WkEuimQc62l2QnArATR8WHP6XBSyamS/zEoEuPPFgG38p9YHf75CB4lgXpTsNyJv
 4NxzPP4MaXpM0ESjLZVXx2TM8LA79fF2OPLap3y2I3ayodAMMalSpWhz6srRIKZ5
 b2omKyVFKlVeYzwRB+EjioKG1JrM443efyS9GEZhC75XEKuQ9mio6A2uSOg8fe4O
 h3JPTfwcXKWMmcwg3rhptmCAQNkzrNFBa3k62/mkf3fzilJHsr3Koby3G73lsgww
 8AISMEeh8nOG4k60CoqMQisG6vOG8fUaI8migwcipbbmbIcDU4+v4A9zPVO+gBDk
 c5zF1qMrvCITsofX5LBg99e59+I23htk/rOBae+ShSyetYx7BC1jM7qLSz364lnV
 PEHEeCKOCJlLQYMdOTM3dOExcb2YI3bnG7vqKShCSwvytiidVb1O95uUCP1/rOH0
 jkW20EPsRqp3qW2RqSaDwhvMpfl/cFnR9dZmkHjM4ApGhmGEUGuI9WGHUw/S6EN9
 NTR/POdrKwRKs9rKAlccHywFOMttXWbtSTHpDPWBA38YKsOMFUc61JHSYuPd1kLV
 EOImT8cymuMhAivtImipd17CY4DND6XhEpTFtm8Opb5a9P+fK3paKlqPlT23n6BM
 83FIPPqQT4y5buD9hCy+bf3CkAcBVRAUfOyPaVWvXR+Ihcl6UhTXSOyzXI/2JNmX
 lfTOtpVgJImrr0/jn/5yrNxJNoeVsAVv4gP7YarWBALcnn3QU2FLDKCAVG5OtSCr
 XkQ6hy5Sr4CdPbHU0SDRtjfiS1vu44Gdi9Uck/wMi9D6LFIq1k0OG2ozXPSgiN/S
 cvHTlF9oD/E2DyjABM4IbVL+KutEx27/jBN9sZJlPGhVibBlQfSnTkFqk2sC80rB
 fF9i0QxiqmT+bZ4iK8AHsEcbc2bv4ia3qxUMy6wPUmQx0MFqtmf+B9MauJH88qnQ
 LjfrmTm0KStILsux3cPTCIXzQw4at9E7Dihxoti973Buh9HX2lxLeHCuiuxmUmtO
 XAwshmHDC7W/GLq5uN3k4gq9pa5N5RA+W0rM4iUiRq//GkFUwDo9xFGD+lKK4D6H
 YqZ/cQwnLNOKtwyCIyL2fBgj/7TtaIy5PhQJDvPvcpKBSUzUGzcqnCWgy9v/Ma4z
 pShhe8G7UDPTiYWI5iYM5SuUanskHnndU2hCdOoD/ZMkoyKvKbQ2/AyinXuA02v9
 jyrFRbbHFRV/Me/x+DUn7dJpXRsfC9lmtBV1Ev6K9nJGJU4tcYkAwxkNrFEW9q9f
 q8n5/jkDUHvQGgEXvKBJCRPJ+HC+kninVv4+eSeWAWGqRFQaedCqPVdxXprqfSvI
 LbxhAMW6XspZPREqw8HLxgZD9siKXEYf26X0FLjX3INoolHedQvWD1fn2DgcZAsJ
 R2714XbSnjErB4/wkw/4bQOzvu2fSBEI7Cnf7WbZzv8gft9r8sNJ7xc1F7WZtbkS
 CrMmTojBA7av5aS8csHiqpxpEaCgFARtiapobhnqElEMRAfwXej9fXVHmlDgeamB
 Zt8pIOYrjNjj3eDjcA4hI5iuRh+/UG3djIG7sCx2COFvbRiAMfXw+WJaMyhMWgzi
 b8R4LF+a2mc6TOCZFaP2ByfOw5T/uz3pL0pw5+fn44+W5fND2UQ1fOS7KDcafQMd
 8ca+pzpm1EBLglbKVdDCu3Zj3JbwdibJz3447v6G5dPPY4JZIoH8FLLXbjQzSWil
 qYCPcjo26QgmuxyJdCHVpWSMUFzUEXgXtCmaFDjvp5uGReL09WlBSGa+GSe0plDW
 dCtTs2VCGqLA45yorjZXKDPSFjS3iQmpFQDkldlAoo4w/d+PKIyy52UiBawXuX96
 sBrxes+jTT8ho2azOAy0BfY+TJwg94YUhrBwksmzUHUaQA7gUcROIwpvXDQqLLLi
 YPTHaqhplKPRbRHGYOy4PHougSEf2oUsieDxG6XfVTjh3zXCgBST3/zlnFSLXxFD
 l8TPdu0UNm1+n9amgfS7c1a0xu/P1WO8cjZOBSZqeN2ehkn6mLSR6wqj+eVZxe4G
 x0cDXTC6+13kBJG81RVqfxUWBOOwAg56rFKLhkMdbhhrPWtcwYU+dEagkz9oG/YA
 gQ0AAz8csvTSS3LR723k+PePv+7EKhnGKgNDKJUVSfaz9bf9jXXqF6w0ytimvdbU
 lStm/jbgkRKhrqQW6DfLPd9PviqwsaiqMhmf/HHhCDv1NAshVuV8G23FUYCoYJz4
 KlDyXn02OqSsqx4JhRBRQyo2h1+CVSaEuaK4QyAQfWKhwYxeqrkI9oy5pq3uKJGJ
 kDdDbJfFRDk08FQ5NuZ70zCQGiYM86YYRSLz+5dXYUzwZ/vsRYrDx1XCNqMnz3Y5
 vUyYw7EtKP4XTlWK6ZfIJ+NhO4wifHVxQGmXbvjVBD2VEb1nJSZHg9ojzLqrdcNx
 ckTsfV/YzIkUO0ENmRda2neEQFTGEV6NtSGIC77tHmygkLJ5IqKBQOvjZ+mhgehp
 jwQT+MmRc9iItYjUnT/pZgz85WNOS526KP/VKIF5abReAg6+xJ3GVYxpjO6qMjWi
 SXWoB3PAIyhCxrgVc2tHKcGUT9QmnUCahyG//QqzPmROxauv0V8R8DdLhaB+UlIZ
 Dxv4HQTZwdcqWntqILJHpw8YH7NLPT2VaqAUtsCpzkMLQS4ZQH+kQv6LlcmgSWOs
 1SDgII8h2oNrGG4/pjMv4N1s43a4E5AmFMFDM4Cv+P8I0MzwhagOFS9d+s3144XF
 1jdshZ9X/bemjnv2nK+84bued6t28+te8BQdGIKYvajjlpmcB/xV6hrbkIrStxty
 tuThisoMY98yv3RUBU63J4hUOINNzllHWDubRvzBD3eZeEeaW5tEeGL9/aUrNmt3
 3TgYlmIvm1wBM6kzZ8JH+RS/yf3TXGimNLwTtiBEVHnbWIbqLhvIsCpKB37n9UNo
 c8dLxNRPWt9G+XDRvi43VB4d54K7BETfs2fDz12pI/WlDaqxBZyHlnjpo3R1410T
 NmzkjBrBENr9RRzuSNjkLvDNXxr969VTgVvILkp8cH5IewaMiwdTqWApzSkLQ5Gq
 LNqKOnDGIMx4v5RPQrhI6s9RqhFHthQsS8hM+VMuOSGoiC0aYK3s6TpN536n/yIP
 jjPMDhqBk/DWBO+N7kg/zeF8KvMi4dT3lJc0wMiYAwtqA8gHyZNYZhpg+eyjzswZ
 F7CDW0++NDsHjkrVoQbdyuGAbXg1tfo9PHagLJcI1O/n2lYSSXHlsxCxps6ih8WV
 Wbu+07JCGQ+AEfGrqSOHBF5mtj/iWAH3xQChg6atLw3uem39XUIkYnr+XC4bMmRD
 U8z97xeIFrmd7JhbxtKi5AoHlDrfRkSn2ZGD1o1ouGAwhRpJUpNjcMv55UzokyCe
 cEUSsg5ElfK2f6+5ZyVWHPsJ9FxfjDM/AUyQlViveJxRCSz2LpJ63hTa9LyTBhqW
 QZQ6775FHtW837lrmheRFoXp27ak8y4JnRX1VPIpqAzwxaVg/oX+jkDJkUce4jAV
 2dDKYz+VnjIv82Farx9wMaW3K5y5zH5QDr+x15oYmez8Sb2wxmGTwZfNP5eZagmd
 hP5NX5Q21qUk3XX1SDnXtIRIjgLIJrygfU6hgLdYTPX44Tnc8JtW6TQpEbuaFy5J
 TCqzVuxzoWzyEDK1DKZPINxSs2GsLIadV7Q4Wkk9beK6RwchMmNPQJBmQZ7uJhO0
 D0PvksiYV2bTLojuhW1IYUwe3xHroCLOzdPItH/Lq0eCSpLtXifUC1xtcPmZLCrB
 SwOU0/bwK82CVoKcHlhOkzmSP/KvE4CmYrYvuOPnv47gfZcN1EkmDEBYTTsGI4FS
 fjtwcGjEiAJTuCIySWJaJwpIdJ7wmjgXbi1kkHcTYqRTXLqhWiesF2omap/Nzcmx
 6fY43ANcIBMOeFGO5pcRfpLzX611icSeulNRQlFqb+OQtFeYRHc7/I6oYsguOvmZ
 j6JTQzQOND25+jcv1WsW3myBjryHgbm1UlTcxH4xwiczNXoHJ8va+g+SBjP+oux9
 B+KZTpBHG9YfkPFOHUMTd42lZtO8G9KloJchIPfZdNiheIIHQGd3cX1rzhNrxTK/
 1725LUSciSacXRp3wD03zMkGdXXfn/iv0shpayxfBpbABuM3vDjOLD9bA8D2jfTN
 MJe9pyb9ek6y8yctWncA9Ii2mYcC438yL+0LZ+SD1PVyOB2m5xQ0Gq3YXvJOvtIU
 a1wEfTEn8DddBJNk6jn1RAIwJzuC2Doy1O5Wskybwh6UXzbeV8zuP+7q/wupn1Ox
 yLTXN/pB+lAu9C6vYnW52nQrE9nZLO3Kwh8/yU8uUDYs0i4qeUwMCeQWjn5nXc05
 gNDz+Zb9bKCS60azbldfCF01MOixCWNVGeSb66Zz2asSKCWj7DHQ4LedTRZzoZeR
 =aq7v
 -----END PGP MESSAGE-----
EOF


---

### Step 5.5 Create your secure build server custom configuration

Anybody who uses the secure build server feature of HPVS will use the same Secure Build Server image, but that image is deployed to that person's individual HPVS instance, and that individual instance is uniquely customized through a configuration file. This ensures that your Secure Build Server can only build the application you specify, and push it to the registry that you specify in the configuration file which is created by the below code cell.  Don't run it yet!  You have to tailor a few settings in the code cell below before you run it:

1. For the value of the _DOCKER_REPO_ key, replace `YOUR-CR-NAMESPACE`  with the name of the IBM Container Registry namespace that you either created earlier in *Step 4.2* of the lab, or that you already owned.

2. For *both*:

    a. _DOCKER_PASSWORD_

    b. _DOCKER_RO_PASSWORD_

 replace `YOUR-IAM-API-KEY-VALUE` with the API Key value you created earlier in the lab in *Step 3.1*. 

3. For these three keys:

    a. _DOCKER_BASE_SERVER_

    b. _DOCKER_PUSH_SERVER_
    
    c. _DOCKER_CONTENT_TRUST_PUSH_SERVER_

replace `YOUR-ICCR-REGISTRY` with `us.icr.io` if you are using the _us-south_ for your IBM Cloud Container Registry region, or with `au.icr.io` if you are using the _ap-south_ IBM Cloud Container Registry region, or with `de.icr.io` if you are using the _eu-central_ IBM Cloud Container Registry region.  Refer back to *Step 4.1* if you forgot which region you used.

4. For the value of the _ACCESS_TOKEN_ key, which is within the _ARG_ key at the bottom, replace `YOUR-GITHUB-ACCESS-TOKEN` with the value of the GitHub personal access token that you created in _Step 2.5_.

Leave all of the other values unchanged.

Once you have made these three substitutions in the below code cell, run the code cell.

**Note:** This code cell also does not produce any output within the Jupyter Notebook for the same reason as the previous code cell.

In [ ]:
%%bash
cat <<EOF > sbs-config.json
{
    "CICD_PUBLIC_IP": "",
    "CICD_PORT": "443",
    "IMAGE_TAG": "1.3.0",
    "CONTAINER_NAME": "SBContainer",
    "GITHUB_KEY_FILE": "~/.ssh/id_rsa",
    "GITHUB_URL": "git@github.com:IBM/secure-bitcoin-wallet.git",
    "GITHUB_BRANCH": "security-fixes",
    "IMAGE_TAG_PREFIX": "s390x-v1",
    "REPO_ID": "sbs",
    "DOCKER_REPO": "YOUR-CR-NAMESPACE/secure-bitcoin-wallet",
    "DOCKER_USER": "iamapikey",
    "DOCKER_PASSWORD": "YOUR-IAM-API-KEY-VALUE",
    "DOCKER_RO_USER": "iamapikey",
    "DOCKER_RO_PASSWORD": "YOUR-IAM-API-KEY-VALUE",
    "DOCKER_CONTENT_TRUST_BASE": "False",
    "DOCKER_CONTENT_TRUST_BASE_SERVER": "",
    "DOCKER_BASE_SERVER": "YOUR-ICCR-REGISTRY",
    "DOCKER_PUSH_SERVER": "YOUR-ICCR-REGISTRY",
    "DOCKER_CONTENT_TRUST_PUSH_SERVER": "https://YOUR-ICCR-REGISTRY:4443",
    "ENV_WHITELIST":  ["ZHSM", "APIKEY", "INSTANCE_ID", "IAM_ENDPOINT"],
    "ARG": {
      "ACCESS_TOKEN": "YOUR-GITHUB-ACCESS-TOKEN"
    }
  }
EOF


---

### Step 5.6 Verify your custom configuration file

You can run the code cell below to display the contents of the file you just created to ensure it looks good.  If you made a typo, you can probably fix it by going back to the above code cell and running it again. 

The code cell below pipes the output of the _cat_ command to a utility called _jq_ (for _JSON query_) which expects well-formed JSON (JavaScript Object Notation) data. If you made a mistake while updating the file that causes a JSON syntax error, the _jq_ utility will detect it and print an error message.  

If you put in a wrong value that doesn't cause a JSON syntax error, _jq_ will not catch that, so inspect the output of the code cell carefully.  All variables that you are required to change started with _YOUR_, so it isn't a bad idea to look for the word YOUR in the output—you shouldn't find it!

In [ ]:
! cat sbs-config.json | jq .

---

### Step 5.7 Generate client certification authority and client certificate

Secure communication between you and your secure build server is obtained by using mutual Transport Layer Security (TLS). TLS is the same security protocol that protects most Web traffic today, but most Web interactions only require the server to be authenticated.  Mutual TLS refers to the situation where not only does the client want to ensure that the server is legitimate, but the server also wants to authenticate the identity of the client.  Your Web browser is constantly receiving server certificates and verifying them while you surf the web. Client certificates are what are used when the server wants to verify the client.  

Create the client certificate and client certification authority (CA) by running the code cell below. You should receive two lines of output, with the second line being `INFO:root:client_certificate: generating client CA and certificate`

In [ ]:
! ./build.py create-client-cert --env sbs-config.json

---

### Step 5.8 Display your environment which shows your client certificate and CA

The next code cell will display the client certificate and the client certification authority in base64-encoding.

**Note:** Keep the output from the below code cell.  You will be copying most of its output into your clipboard in a subsequent step.

If you already know about _base64-encoding_, go ahead and run the code cell.  If you don't, read the next paragraph if you're curious about what base64-encoding is.  

Base64-encoding is a means of encoding any data, including binary data, into human-readable characters.  It takes groups of 6 bits, converts these six bits into a number from 0 to 63  (2 to the 6th power is 64), and replaces these 6 bits into one of 64 human-readable characters.  What 64 human-readable characters were chosen?  Lowercase alphabetic characters a-z for 26, uppercase alphabetic characters A-Z for 26 more, the digits 0-9 for 10 more, which gives us 62.  Then `+` and `/` were chosen to make 64!  It should be noted that base64-encoding should not be confused with encryption.  It is easy to convert a base64-encoded string back to its original format.


In [ ]:
! ./build.py instance-env --env sbs-config.json

---

### Step 5.9 List your current HPVS instances

Run the following code cell to list your current HPVS instances. If you are new to HPVS, you won't have any.

This lab uses the free version of HPVS instances, and IBM Cloud users are limited to two free instances, and the lab requires two instances. If you are an experienced HPVS user and you have paid HPVS instances, they won't be affected by this lab.  If you are currently using one free HPVS instance and don't want to delete it, talk to your instructor who can help you work around this situation. If you are currently using both free HPVS instances and can't delete at least one of them, then unfortunately this lab will not work unless you are willing to incur the costs of creating chargeable HPVS instances for this lab.

In [ ]:
! ibmcloud hpvs instances

---

### Step 5.10  Create your Secure Build Server with the _ibmcloud hpvs_ CLI

The code cell below has the first part of the `ibmcloud hpvs instance-create` command filled in properly.

**You need to make two changes.**

**First**, you need to change _YOUR-LOCATION_ to one of the locations as explained in the following paragraphs.

If you logged in to _us-east_, then choose one of these three locations: `wdc04` or `wdc06` or `wdc07`

If you logged in to _au-syd_, then choose one of these three locations: `syd01` or `syd04` or `syd05`

If you logged in to _eu-de_, then choose one of these three locations: `fra02` or `fra04` or `fra05`

**Second**, you need to add arguments for the `CLIENT_CRT` and `CLIENT_CA` variables.  All you have to do is copy them from the output from two code cells above, from _Step 5.8_.  Copy the _-e CLIENT_CRT=..._ and the _-e CLIENT_CA=..._  by selecting the output starting from _-e CLIENT_CRT_ and extending to the end of the output. In other words, you are copying almost all of the output from _Step 5.8_ except for the few lines of output that precede the start of the first *-e CLIENT_...* variable that is listed.

I have left a blank line in the code cell below in which you can paste those two excruciatingly long environment variables. Click in the code cell so that your cursor is at the beginning of the blank line 2 of the code cell, and then paste in the variables at that spot.

**Tip:** When trying to copy this long string in the clipboard, when the entire string isn't visible on the screen at once, I have had smoother scrolling when starting from the bottom and then sweeping up to the top to select the text.  Your experience may differ.

Once you have tailored the code cell per these instructions, run it.  This command may take several seconds to run.

In [ ]:
! ibmcloud hpvs instance-create SBContainer lite-s YOUR-LOCATION --rd-path "secure_build.asc" -i 1.3.0 \

# copy and paste, in the blank line above me, the super long -e CLIENT_CRT=...  -e CLIENT=... from the output of the command \ 
# a couple cells above.  you can leave me here because I'm just a comment.

---

### Step 5.11 Check the status of your Secure Build Server HPVS instance

It will take several minutes for your instance to be built. The command output from the previous code cell in *Step 5.10* conveniently lists the _ibmcloud hpvs_ command you can use to display the status. Copy the entire `ibmcloud hpvs instance crn:...` command above, which has your unique instance id (shown in the output of the above command, it starts with _crn:_) into the code cell below and run it periodically (about once a minute) until the instance is successfully created. 

**Tip:** Your instance is successfully created when the value of the _Cloud state_ field of the output is _active_. For the first several minutes you will see that the _Cloud state_ is _provisioning_ and that most of the values listed below _Cloud state_ are _not available_.  At a certain point, you will see that those values that were _not available_ have been populated, but you may see that the _Cloud state_ remains as _provisioning_ for a little while. Ensure that you wait for the _Cloud state_ to change to _active_ before proceeding with the lab.

In [ ]:
! 
# copy and paste above the entire 'ibmcloud hpvs instance crn:.....' commmand from the \
# output from the previous command, and run it periodically (every minute or so) until 'Cloud state' is active.

---

## Create your sample Secure Bitcoin Wallet Application BYOI image ##

### Step 6.1 View your custom configuration file again.

The code cell below will show the current state of your secure build configuration file.  You have already looked at this file from an earlier code cell, but we're repeating it here so that you can observe the key thing I want you to note from the output—that the IP address value of the _CICD_PUBLIC_IP_ key is blank.

In [ ]:
! cat sbs-config.json | jq .

---

### Step 6.2 Add the public IP address of your secure build server to your custom configuration file

Now that your secure build server instance is running, make a note or copy the IP address shown in the *Public IP address* field from the command output from *Step 5.11*. The code cell below uses an awkward-looking, yet simple _sed_ command to add the IP address in the proper spot.  

**Editing the below code cell is required:** Replace the _xx.xx.xx.xx_ string below with the Public IP address of your secure build container, then run the code cell. Make sure you leave the double-quotes surrounding _xx.xx.xx.xx_ in place. You should not see any output from this code cell:

In [ ]:
! sed -i -e 's/\"CICD_PUBLIC_IP\"\: \"\"/\"CICD_PUBLIC_IP\"\: \"xx.xx.xx.xx\"/' sbs-config.json

---

### Step 6.3 Verify that you added your public IP address correctly

Run the code cell below to repeat the command that displays your configuration file. Ensure that your public IP address was added properly.  Instead of _"CICD_PUBLIC_IP": "",_ the line should look like _"CICD_PUBLIC_IP": "xx.xx.xx.xx",_ but with your Secure Build Server instance's public IP address instead of _xx.xx.xx.xx_.

In [ ]:
! cat sbs-config.json | jq .

---

### Step 6.4 Check the status of your Secure Build Server

Run the following command to check the status of your Secure Build Server instance. The value of the _status_ field in the output should be empty, i.e., `"status": ""`:

In [ ]:
! ./build.py status --env sbs-config.json --noverify

---

### Step 6.5 Get the server certificate signing request

Run the following code cell to get the server certificate signing request (CSR):

In [ ]:
! ./build.py get-server-csr --env sbs-config.json --noverify

---

### Step 6.6 Sign the server CSR

Run the following code cell to sign the server CSR. This command will not produce any output.  You will know it ended if a new number is placed within the square brackets to the left of the command.

In [ ]:
! ./build.py sign-csr --env sbs-config.json

---

### Step 6.7 Post the signed server certificate

To post the signed server certificate to your Secure Build Server instance, run the following code cell. The value of the status field in the output should be _OK_, i.e., `"status": "OK"`:

In [ ]:
! ./build.py post-server-cert --env sbs-config.json --noverify

---

### Step 6.8 Check your Secure Build Server instance status again

To check and verify the status for your Secure Build Server instance, run the following code cell.  The value of the _status_ field in the output should be _restarted nginx_. This is because the nginx server running inside the Secure Build Server was restarted in order to use the server certificate that was just posted in _Step 6.7_.

In [ ]:
! ./build.py status --env sbs-config.json

---

### Step 6.9 Initialize the configuration for your Secure Build Server instance

Run the following code cell to initialize the configuration for your Secure Build Server instance. The status of the output should be `OK`:

In [ ]:
! ./build.py init --env sbs-config.json

---

### Step 6.10 Start the build of your sample secure bitcoin wallet application image

The code cell below will start the build of your application image. The command returns quickly, as soon as it submits the request to start the build.  Your status message from the command output should say `OK: async build started`, which indicates that the build of your image has been started up on your Secure Build Server container. The command does not wait for the build to complete. The build will take several minutes. 

Run the below code cell to start the your build:

In [ ]:
! ./build.py build --env sbs-config.json

---

### Step 6.11 Check the status of your application image build

Check the status of your build by running the code cell below.  You can check it periodically—your build should take less than ten minutes to complete.  You can run this code cell periodically, as well as the command in the subsequent code cell (_Step 6.12_) which displays the logs from the build.  For now, check the status by running the code cell below.  Your build has completed successfully once the output of the code cell has the value of *success* in the *status* field:

In [ ]:
! ./build.py status --env sbs-config.json

In the above code cell, until the build completes, you may see several different statuses, including:

_cleaned up_   This status can appear at the beginning of the build but you may not see it as the status quickly changes to _github cloned_.

_github cloned_  This status appears most of the time as this status remains until the image build is complete, and this takes several minutes.

_image built_  Once the image is built, but before the push of the image into the IBM Cloud Container Registry is complete, you may see this status.

_image pushed_ Once the image has been pushed to the IBM Cloud Container Registry, you may see this status, but usually the build completes very soon afterwards, so you may not see this status, as it should hopefully quickly change to _success_ status.

_success_ Once the secure build process has completed successfully, you will see this status. Once you see this status, you are free to move on (Step 6.13). Remember, you can run the logs periodically (Step 6.12) while you are waiting for this _success_ status. 

---

### Step 6.12 View the logs from your application image build

Your secure image build will produce a lot of output in its log. 

You can check the log of your build by running the below code cell. 

**Jupyter Notebook tip 1:** In some environments this output may be placed in a scrollable window, but in other environments all of the log output may be placed into the notebook, which makes navigating to the rest of the Notebook more difficult.  If this happens to you, please toggle scrolling with `Shift` + `o` (letter o not number 0). This will turn the output into a scrollable window as desired. 

**Jupyter Notebook tip 2:** Additionally, if you ever wish to clear the output of this command from your Jupyter Notebook by clicking within the output and then choosing *Cell -> Current Outputs -> Clear* from the Jupyter Notebook menu above.  (The build logs are retained on the Secure Build Server and can be retrieved at any time).

**Jupyter Notebook tip 3:** Running a code cell clears the output from any previous execution of the code cell, so if you want to view the log more than once, you don't need to clear the output as described in the tip given in the preceding paragraph.

In [ ]:
! ./build.py log --log build --env sbs-config.json

---

### Step 6.13 Download the state image of your Secure Build Server

Run the code cell below to download the state image, which is a backup of the state of your Secure Build Server instance. This can be used to recreate your Secure Build Server if necessary. This backup also contains the logs of any builds you run on your Secure Build Server.

In [ ]:
! ./build.py get-state-image --env sbs-config.json

---

### Step 6.14 Verify the manifest file from your build

The Secure Build Server creates a signed manifest file for each successful build. The manifest file is useful for audit purposes. You can verify the source and integrity of the build and the built image or you can pass the manifest file to an auditor to do so. Run the below code cell to download and verify the manifest file:

In [ ]:
! ./build.py get-manifest --env sbs-config.json  --verify-manifest

The above command downloads the manifest file and verifies the digital signature on it.  In the last line of output from the above code cell you should see *verify=OK* which indicates that the digital signature is valid and is proof that the manifest file has not been tampered with.

---

## Create HPVS instance using the BYOI image that you just built

### Step 7.1 Download the encrypted registration file for the image that you just built

In order to create an HPVS instance using the application image you just built, you must download an encrypted registration file for your application image that was created by the build. This registration file can only be decrypted, and thus used, by your application container image.

**Copy the command in the code cell below (everything but the leading exclamation point) and _run it in your terminal tab_**. As part of this command, you will be prompted to enter a passphrase three times—the first two times to set the passphrase, and then the key is used to sign the registration file, so you are asked to enter the passphrase you just set.  You can set the passphrase to whatever you want.  For this lab, you only need to remember it for the duration of this command—it's not used in any subsequent commands in the lab.

Note the value chosen for the *--key-id* argument:  *secure-build-6ecac9a-1*.  This value _6ecac9a_ was chosen as this is a hash of the image we built and is appended to the end of the tag name. 

If you look at your command output a few code cells above, from _Step 6.11_, where you checked on your build status, you can see this hash at the end of the value of the *image_tag* key, e.g.,  `"image_tag" : "s390x-v1-6ecac9a"`. 

The *--key-id* value can be anything, but putting this hash value as part of the name helps to keep from confusing different keys if you have multiple keys.

**Note:** This command won't work if run in the code cell because of the need to type in a passphrase.  That is why you have to run it in the _Terminal_ tab.


In [ ]:
! cd /home/ibmuser/code/ibm-cloud-hpvs-sbs-byoi-lab/secure-build-cli && ./build.py get-config-json --env sbs-config.json --key-id secure-build-6ecac9a-1

**Jupyter Notebook tip:** If you accidentally ran the above code cell within this notebook instead of copying it into the terminal tab and running it there, it may never finish because it needs user input.  If the square bracket to the left of the code cell has an asterisk, you can interrupt the command  by choosing _Kernel-> Interrupt_ from the Jupyter Notebook menu above. 

---

### Step 7.2 Launch your BYOI application as an HPVS instance

Make the change described here to the below code cell and then run the code cell below to create your HPVS instance using your BYOI image that you just built.  The value for the `-i` argument must be the value of the *image_tag* key in the output of *Step 6.11* which checked your build status.

**Tailoring the below code cell before your run it:** You need to make two changes:

1. You need to change _YOUR-LOCATION_ to one of the locations as explained in the following paragraphs.

    If you logged in to _us-east_, then choose one of these three locations: `wdc04` or `wdc06` or `wdc07`

    If you logged in to _au-syd_, then choose one of these three locations: `syd01` or `syd04` or `syd05`

    If you logged in to _eu-de_, then choose one of these three locations: `fra02` or `fra04` or `fra05`
    

2. Ask your instructor for the APIKEY which allows access to the instructor-provided Hyper Protect Crypto Services instance, and replace _YOUR-INSTRUCTOR-PROVIDED-APIKEY_ with the value that the instructor gives you.

In [ ]:
! ibmcloud hpvs instance-create securewallet lite-s YOUR-LOCATION --rd-path sbs.enc -i s390x-v1-6ecac9a \
-e "ZHSM"="ep11.us-south.hs-crypto.cloud.ibm.com:8367" \
-e "APIKEY"="YOUR-INSTRUCTOR-PROVIDED-APIKEY"

---

### Step 7.3 Check the status of your BYOI application instance

It will take several minutes for your instance to be built. The command output from the previous code cell in *Step 7.2* conveniently lists the _ibmcloud hpvs_ command you can use to display the status. Copy the entire `ibmcloud hpvs instance crn:...` command above, which has your unique instance id (shown in the output of the above command, it starts with _crn:_) into the code cell below and run it periodically (about once a minute) until the instance is successfully created. 

**Tip:** Your instance is successfully created when the value of the _Cloud state_ field of the output is _active_. For the first several minutes you will see that the _Cloud state_ is _provisioning_ and that most of the values listed below _Cloud state_ are _not available_.  At a certain point, you will see that those values that were _not available_ have been populated, but you may see that the _Cloud state_ remains as _provisioning_ for a little while. Ensure that you wait for the _Cloud state_ to change to _active_ before proceeding with the lab.

In [ ]:
! 
# copy and paste above the entire 'ibmcloud hpvs instance crn:.....' commmand from the \
# output from the previous command, and run it periodically (every minute or so) until it is running.

---

### Step 7.4 Edit this markdown cell to create a clickable link to get to your sample application

Once your instance is active, note the value of the _Public IP address_ field from the output from _Step 7.3_.  

Double-click anywhere within this paragraph to go into _edit_ mode of this markdown cell.  Then, in the final line of this markdown cell, change `xx.xx.xx.xx` to your public IP address.  The actual URL is on the right side of the line (between parentheses). The text of the link is on the left side of the line in between square brackets.  You may as well change both occurrences of `xx.xx.xx.xx` for clarity, but the one that matters is the link on the right side.   (If you don't see both sides you haven't succesfully got into edit mode—keep trying or ask an instructor for help).  After you have made the change, run this markdown cell by clicking *Run* in the Jupyter Notebook menu above. This will render the markdown and present you with the properly formatted link. Click the link and you will bring up your sample secure bitcoin wallet application.

[Change the xx.xx.xx.xx to your public IP and then click here](https://xx.xx.xx.xx/electrum)

## Running the Secure Bitcoin Wallet Application in your BYOI HPVS instance

### Overview

This application is a secure [Electrum](https://electrum.org/#home) Bitcoin wallet found at [this GitHub repository](https://github.com/IBM/secure-bitcoin-wallet). You can use it to store the identities necessary to transact on the Bitcoin public blockchain network. While this version of the wallet is just a proof of concept, running this type of wallet on Hyper Protect enables you to protect your digital assets from bad actors such as malicious server administrators.

*This Electrum Bitcoin wallet functions regularly, while inheriting the security benefits of the Hyper Protect Virtual Server platform. This ensures that a legitimate version of the application is running (Secure Image Build process) and that administrators cannot compromise the application (Hyper Protect Virtual Server deployment).*

### Step 8.1 Account Registration

Registering a fake account for lab purposes.

1. Choose `Register` since you won't have an existing account on the newly deployed server.

    ![Secure Bitcoin Wallet Welcome](Bitcoin_Wallet_Images/Bitcoin_Wallet_Homepage.png)

2. Enter the registration details you desire.

    ![User Registration](Bitcoin_Wallet_Images/Wallet_register_info.png)


*Do not worry about the information you enter on the registration page (shown below) as it won't map to anything critical to the real world.  You just need your password to log back in if you log out of the application.*

### Step 8.2 Wallet initialization

It's time to set up your Secure Electrum Bitcoin wallet itself!

1. Bring up the wallet screen

    ![Wallet Start](Bitcoin_Wallet_Images/Click_Wallet_Start.png)

2. Enter in a `password` for your wallet and click `Create Wallet`

    ![Create Wallet](Bitcoin_Wallet_Images/Create_Wallet.png)

3. Load your newly created wallet to enable it

    ![Load Wallet](Bitcoin_Wallet_Images/Load_Wallet.png)

4. Reload the page.

    ![Initialized Wallet](Bitcoin_Wallet_Images/Reload_new_wallet.png)

5. Your wallet is ready for use in all of its glory

    ![Locked and Loaded Wallet](Bitcoin_Wallet_Images/Loaded_Wallet_Time.png)

### Step 8.3 Receiving Bitcoins

A wallet without currency is blasphemy, so it's time to fill it on up with fake bitcoins.

1. Go to the `Receive` section of the wallet and get the bitcoin wallet address.  

    ![Wallet receiving bitcoin](Bitcoin_Wallet_Images/Receiving_bitcoin.png)

    *This is the address whose alloted bitcoins you lay claim to.*

2. Open [this bitcoin test faucet site](https://bitcoinfaucet.uo1.net/send.php) [click on the link] and use it to transfer bitcoin value to the `Receiving address` in your wallet **from step 1 above**.

    ![Transfer Bitcoins](Bitcoin_Wallet_Images/Send_bitcoins_to_wallet.png)

    *Instead of transacting on the "real" bitcoin blockchain network, we are using a [testnet](https://medium.com/myetherwallet/understanding-blockchain-changes-testnets-and-mainnets-c2171a8e835f). This means that the network participants agree that the assets on the network have no value. Testnets are used for testing changes to the network, testing applications and introducing new users to the technical aspects of network. There are testnets for many of the public blockchain networks including of course bitcoin and ethereum. Test faucets (like the one we are using) act as distributors of tokens to other users on the testnet.*

3. Switch back to your wallet tab and go to the `History` section and then reload the page.

    ![Check History](Bitcoin_Wallet_Images/Check_History_Receive_Transaction.png)

4. You should now see your wallet address as the recipient of a `pending transaction` from the bitcoin testnet faucet.

    ![First Transaction History](Bitcoin_Wallet_Images/Transaction_History_no_confirmations.png)

5. After clicking for more information you get a detailed transaction overview

    ![Receive Transaction 0 Confirmations](Bitcoin_Wallet_Images/0_confirmations_block_details.png)

    *The number of [confirmations](https://en.bitcoin.it/wiki/Confirmation) a block receives is the number of blocks deep the block that has included the blockchain transaction is. In other words, `1 confirmation` means that the current head block includes the transaction. `2 confirmations` means that the current head blocks parent block contains the transaction and so on. The deeper the transaction is (more confirmations it has) the more mining power it would take to "reverse history" and undo the transaction.*

    Since there are `0 confirmations` in the picture above, it means that the transaction is not yet included in a block. This is why it has no timestamp yet and it's block number is listed as `0`.

6. After waiting anywhere from a minute to a few minutes (depending on the current testnet traffic) you should see a confirmation on your transaction.

    *You will need to keep reloading the page for it to update. It doesn't update automatically.*

    ![1 Confirmation Received](Bitcoin_Wallet_Images/1_Confirmation_Received.png)

    There is now a timestamp and a block number! :dollar:

    *As time passes more confirmations will accumulate. You can check on the `confirmations` again at the end of the lab to show yourself this (Given enough time has passed for one or more additional confirmation(s) to occur).*

### Step 8.4 Returning Bitcoins

Since you are feeling charitable you decide to transfer your testnet value back to the faucet's address.

1. Go back to [the faucet site](https://bitcoinfaucet.uo1.net/send.php) and get its bitcoin address

    ![Faucet Address](Bitcoin_Wallet_Images/testnet_faucet_address.png)

    *This address is here for easy copy and paste (ensure this matches what the application shows in the browser before using this, as the bitcoin address they use for returning funds changes occasionally):

    ``` bash
    tb1qm5tfegjevj27yvvna9elym9lnzcf0zraxgl8z2
    ```

2. Send bitcoins to faucet using `Send` section of wallet

    ![Send Bitcoins to Faucet](Bitcoin_Wallet_Images/Send_bitcoins_back_to_faucet.png)

3. Once sent you will see the `Signed transaction` information filled out on your wallet page.

    ![Bitcoins Sent Back to Faucet](Bitcoin_Wallet_Images/bitcoin_send_transaction_output.png)

4. You can go back to the [bitcoin faucet site](https://bitcoinfaucet.uo1.net/send.php) and see the transaction as green there with a sender of your address and a transaction ID matching the one you sent.

    ![Bitcoins returned to faucet](Bitcoin_Wallet_Images/transaction_giving_coins_back_to_faucet.png)

5. You can view both the initial received transaction and the newly sent transaction in the `History` tab of your bitcoin wallet.

    *You have to keep reloading the page for it to update with new confirmations. If you wait long enough, eventually these transactions will gain confirmations as in the picture below*

    ![Both transactions with 1 confirmation](Bitcoin_Wallet_Images/both_confirmed_transactions.png)

6. You can continue to play around with the electrum wallet before moving on.

### Summary

You have participated in the bitcoin testnet using a securely built electrum bitcoin wallet powered by a Hyper Protect Virtual Server. Congratulations!!! 

Now, it's time to clean up..

---

## Clean up when you are done with the lab

This section describes important cleanup steps to perform when you have created the lab. It will guide you through deleting the following artifacts that you created during the lab:

1. The Secure Build Server HPVS instance you created
2. The sample Secure Bitcoin Wallet Application HPVS instance you created
3. The Secure Bitcoin Wallet Application container image that you created for this lab
4. The IBM Container Registry namespace that you created for this lab
5. The IBM Cloud IAM Key you created for this lab
6. The ssh key you created for this lab


---

### Step 9.1 List your hpvs instances

Run the code cell below to list your HPVS instances:

In [ ]:
! ibmcloud hpvs instances

---

### Step 9.2 Delete your Secure Build Server HPVS instance

In the output from _Step 9.1_, find your instance named *SBContainer*, and copy its CRN (_Cloud Resource Name_). Paste it at the end of the `ibmcloud hpvs instance-delete ` command in the below code cell and then run it.

**Note:** Make sure that your paste operation leaves a space between the end of your instance's CRN and the _#_ character the begins the comment in the code cell. 

In [ ]:
! ibmcloud hpvs instance-delete --force     # <- paste your full CRN name to the left of the pound sign

---

### Step 9.3 Delete your sample Secure Bitcoin Wallet Application HPVS instance

In the output from _Step 9.1_, find your instance named _securewallet_ and copy its CRN value and paste it at the end of the _ibmcloud hpvs instance-delete_ command in the below code cell and then run it.

In [ ]:
! ibmcloud hpvs instance-delete --force    # <- paste your full CRN name to the left of the pound sign

---

### Step 9.4 List your IBM Cloud Container Registry images

Run the following code cell to list your IBM Cloud Container Registry images:

In [ ]:
! ibmcloud cr images

---

### Step 9.5 Delete the container image for the sample secure bitcoin wallet application

From the output from _Step 9.4_, copy and paste the Repository name (first column) of the image you built for this lab, and paste it into the code cell below after `ibmcloud cr image-rm ` and then run the code cell.  This will remove your image from the IBM Cloud Container Registry.

In [ ]:
! ibmcloud cr image-rm    # <- paste your repository name after 'image -rm '

---

### Step 9.6 Verify that your image has been deleted

You can rerun the `ibmcloud cr images` command to verify that you deleted the Secure Bitcoin Wallet Application image. It should not show up in the output from the below code cell:

In [ ]:
! ibmcloud cr images

---

### Step 9.7 List your IBM Cloud Container Registry namespaces

**Note:** If you used an existing namespace and wish to keep it, skip this step and skip _Step 9.8_.

Run the following code cell to list your IBM Cloud Container Registry namespaces:

In [ ]:
! ibmcloud cr namespaces

---

### Step 9.8 Delete the IBM Cloud Container Registry namespace that you created for this lab


Type, or copy and paste from the output from _Step 9.7_, the namespace you created for this lab to the end of the code cell below and then run it.

In [ ]:
! ibmcloud cr namespace-rm --force         # <- paste the name of the namespace you want to delete

---

### Step 9.9 List your IBM Cloud IAM Keys

Run the code cell below to list the IAM Keys associated with your account.

In [ ]:
! ibmcloud iam api-keys

---

### Step 9.10 Delete the IBM Cloud IAM Key you created for this lab

If you used the lab default name of _myapikey_ then the below code cell is ready to run. If you used a different name, find the key that you created for this lab, and copy and paste either the _Name_ (first column) or the ID (last column) into the code cell below, replacing _myapikey_.  Run the code cell in order to delete your IAM key.

In [ ]:
! ibmcloud iam api-key-delete --force  myapikey

---

### Step 9.11 Verify that you deleted the IAM Key you created for the lab

List your API keys again by running the code cell below and ensure that the key you created for this lab is no longer listed in the output:

In [ ]:
! ibmcloud iam api-keys

---

### Step 9.12 Remove the ssh key you created for this lab from your GitHub account

Delete the ssh key you added for the lab from your GitHub account by following steps 1-3 of [these instructions](https://docs.github.com/en/github/authenticating-to-github/reviewing-your-ssh-keys), choosing the _Delete_ button for the ssh key you want to get rid of.  If you were an existing GitHub user prior to this lab, be careful not to delete any other ssh keys you may have added for other purposes.

---

### Step 9.13 Remove the GitHub personal access token you created for this lab from your GitHub account

You should still be logged in to GitHub from the previous step, _Step 9.12_.  Click on your picture or avatar for your account and navigate to _Settings->Developer Settings->Personal access tokens_ and delete the personal access token that you created in _Step 2.5_ of this lab.  Ask an instructor for help if you have difficulty deleting this.

---

## You have reached the end of the lab!

Thank you for taking this lab, and please feel free to provide feedback, either by opening a GitHub issue on the lab repo at https://github.com/ibm-wsc/ibm-cloud-hpvs-sbs-byoi-lab or by sending an email directly to the primary author of the lab, Barry Silliman at "silliman at us dot ibm dot com"  (the email spammers will never figure out my real email address from that, but you will).

---

## Acknowledgements

My esteemed colleagues Jin VanStee and Garrett Woodworth also made significant contributions to this lab, but I take responsibility for any defects or shortcomings in the lab. -  Barry Silliman

## References

This lab is based largely on publicly available material on IBM's website, especially [Tutorial: Using Secure Build Server with a digital wallet](https://cloud.ibm.com/docs/hp-virtual-servers?topic=hp-virtual-servers-tutorial_secure_build_server)

Additional information about using the Secure Build Server CLI can be found [here](https://github.com/ibm-hyper-protect/secure-build-cli)  

Here is the [GitHub repo for the sample secure bitcoin wallet application](https://github.com/IBM/secure-bitcoin-wallet)